In [89]:
from numba import njit, prange
from typing import Literal, NewType, Sequence
from enum import Enum
from numpy import empty, argsort
import numpy as np
from random import choice, seed

In [ ]:
def encode_endings(endings):
	'''
	b = 0
	'x' = 1
	'o' = 2
	'neutral' = 3
	'positive' = 4
	'negative' = 5
	'''
	encoded_type = '10u1'
	endings_count = len(endings)
	result = empty(endings_count, dtype=encoded_type)
	for i in range(endings_count):
		ending = endings[i]
		temp = []
		for figure in ending[:9]:
			if figure == 'b':
				temp.append(0)
			elif figure == 'x':
				temp.append(1)
			elif figure == 'o':
				temp.append(2)

		if ending[9] == 'neutral':
			temp.append(3)
		elif ending[9] == 'positive':
			temp.append(4)
		elif ending[9] == 'negative':
			temp.append(5)
		result[i] = temp
	return result

def read_endings():
	source = np.genfromtxt('tic-tac-toe.data', delimiter=',', dtype='U30')
	for ending in source:
		if ending[9] == 'negative' and 'b' not in ending:
			ending[9] = 'neutral'
	return encode_endings(source)


endings = read_endings()
print(endings)

In [91]:
@njit()
def get_situation(x_moves, o_moves):
	return np.array([1 if (i in x_moves) else 2 if (i in o_moves) else 0 for i in range(9)])

@njit()
def get_ending_distance(situation, ending, player_is_x):
	ending_distance = 0
	for i in range(9):
		if situation[i] != ending[i]:
			if situation[i] == 0:
				if player_is_x and ending[i] == 1:
					ending_distance += 1
				if not player_is_x and ending[i] == 2:
					ending_distance += 1
			else:
				return -1
	return ending_distance

@njit()
def get_move_endings(result_endings, move, situation, player_is_x):
	move_situation = [situation[i] if i != move else 1 if player_is_x else 2 for i in range(9)]
	endings_result_count = len(result_endings)
	move_endings_count = 0
	move_endings = empty((endings_result_count, 2), dtype='u1')
	for e_i in range(endings_result_count):
		ending = result_endings[e_i]
		ending_situation = ending[2:11]
		dist = get_ending_distance(move_situation, ending_situation, player_is_x)
		if dist == -1:
			continue
		result = ending[1]
		move_endings[move_endings_count, :] = [dist, result]
		move_endings_count += 1
	return move_endings[:move_endings_count]


In [92]:
@njit()
def analyse(endings, player_is_x, x_moves, o_moves, allowed_moves):
	endings_result_count = 0
	endings_count = len(endings)
	result_endings = np.empty((endings_count, 11), dtype='u1')
	# Уменьшаем число концовок отсекая недостижимые
	# Так же для каждой концовки вычисляем "расстояние"
	situation = get_situation(x_moves, o_moves)
	for i in range(endings_count):
		ending = endings[i]
		ending_distance = get_ending_distance(situation, ending, player_is_x)
		if ending_distance <= 0:
			continue
		# Инвертируем результат если играет "нолик"
		result = ending[9]
		if player_is_x == False:
			if result == 5:
				result = 3
			elif result == 3:
				result = 5
		result_endings[endings_result_count, :] = [ending_distance, result, *ending[:9]]
		endings_result_count += 1
		
	result_endings = result_endings[argsort(result_endings[:endings_result_count,0], kind='mergesort')]
	moves_count = len(allowed_moves)
	analyzed_moves = empty((moves_count, 13), dtype='f4')
	# Группируем концовки по следующему "ходу"
	for m_i in range(moves_count):
		move = allowed_moves[m_i]
		# Отфильтровываем недостижимые концовки
		# [dist, result]
		move_endings = get_move_endings(result_endings, move, situation, player_is_x)
		if len(move_endings) == 0:
			return empty((0, 16), dtype='f4')
		move_endings_count = len(move_endings)
		min_dist = min(move_endings[:, 0])
		dist_count = max(move_endings[:, 0]) - min_dist + 1
		dist_metrics = np.zeros((dist_count, 7), dtype='f4')
		nearest_draw = 4
		nearest_win = 4
		nearest_loose = 4
		# Считаем число ничьих, побед, поражений и общее число игр
		for e_i in range(move_endings_count):
			# dist: 
			# 	1 - 4
			# result: 
			# 	3 - neutral
			# 	4 - positive
			# 	5 - negative
			dist, result = move_endings[e_i]
			d_i = dist - min_dist
			# 3 - 3 = 0; draw_count
			# 4 - 3 = 1; win_count
			# 5 - 3 = 2; loose_count
			#		  3; total_count
			#		  4; draw_rate
			#		  5; win_rate
			#		  6; lose_rate
			if result == 3 and nearest_draw > dist:
				nearest_draw = dist
			elif result == 4 and nearest_win > dist:
				nearest_win = dist
			elif result == 5 and nearest_loose > dist:
				nearest_loose = dist
			metric_i = result - 3
			dist_metrics[d_i, metric_i] += 1
			dist_metrics[d_i, 3] += 1

		# Считаем проценты
		for d_i in range(dist_count):
			for i in range(3):
				dist_metrics[d_i, 4 + i] = dist_metrics[d_i, i] / dist_metrics[d_i, 3] * 100
		
		# Считаем метрики
		# move				0
		# nearest_draw		1
		# nearest_win		2
		# nearest_loose		3

		# max_draw_rate 	4
		# max_win_rate 		5
		# max_loose_rate 	6
		# avg_draw_rate 	7
		# avg_win_rate 		8
		# avg_loose_rate 	9
		# min_draw_rate 	10
		# min_win_rate 		11
		# min_loose_rate 	12
		# total_draw_rate 	13
		# total_win_rate 	14
		# total_loose_rate 	15
		analyzed_moves[m_i, 0] = move
		analyzed_moves[m_i, 1:4] = [nearest_draw, nearest_loose, nearest_win]
		for i in range(3):
			percent = dist_metrics[:, i + 4]
			absolute = dist_metrics[:, i]
			total = dist_metrics[:, 3]
			analyzed_moves[m_i, 3 * 0 + i + 4] = max(percent)
			analyzed_moves[m_i, 3 * 1 + i + 4] = sum(percent) / len(percent)
			analyzed_moves[m_i, 3 * 2 + i + 4] = min(percent)
			analyzed_moves[m_i, 3 * 3 + i + 4] = sum(absolute) / sum(total) * 100
	return analyzed_moves


In [108]:
x_moves = empty(0, dtype='u1')
o_moves = empty(0, dtype='u1')
allowed_moves = np.array([
	0, 1, 2,
	3, 4, 5,
	6, 7, 8,
])

player_is_x = True
seed(0)
for i in prange(9):
	# print(get_situation(x_moves, o_moves), player_is_x)
	result = analyse(endings, player_is_x, x_moves, o_moves, allowed_moves)
	with open('result.txt', 'a+') as file:
		file.write(f'{len(result)}\n')
	m = choice(allowed_moves)
	i, = np.where(allowed_moves == m)
	allowed_moves = np.delete(allowed_moves, i)
	if player_is_x:
		x_moves = np.append(x_moves, [m])
	else:
		o_moves = np.append(o_moves, [m])
	player_is_x = not player_is_x

In [94]:
# import time
# start_time = time.time()

# for i in range(5_000):
# 	if (i + 1) % 100 == 0 or i == 0: 
# 		with open('no_njit_log.log', 'a') as file:
# 			seconds = time.time() - start_time
# 			minutes = int(seconds // 60)
# 			seconds -= minutes * 60
# 			file.write(f'{i+1},{minutes},{int(seconds)}\n')
# 	x_moves = empty(0, dtype='u1')
# 	o_moves = empty(0, dtype='u1')
# 	allowed_moves = [
# 		0, 1, 2,
# 		3, 4, 5,
# 		6, 7, 8,
# 	]
# 	player_is_x = True
# 	for i in range(9):
# 		# print(get_situation(x_moves, o_moves), player_is_x)
# 		result = analyse(endings, player_is_x, x_moves, o_moves, allowed_moves)
# 		with open('result.txt', 'a') as file:
# 			file.write(f'{result}\n')
# 		m = choice(allowed_moves)
# 		allowed_moves.remove(m)
# 		if player_is_x:
# 			x_moves = np.append(x_moves, [m])
# 		else:
# 			o_moves = np.append(o_moves, [m])
# 		player_is_x = not player_is_x

In [95]:
# print(*[range(1, 3)] * 3)

In [26]:
import numpy as np

# nearest_draw		1
# nearest_win		2
# nearest_loose		3

# max_draw_rate 	4
# max_win_rate 		5
# max_loose_rate 	6

# avg_draw_rate 	7
# avg_win_rate 		8
# avg_loose_rate 	9

# min_draw_rate 	10
# min_win_rate 		11
# min_loose_rate 	12

# total_draw_rate 	13
# total_win_rate 	14
# total_loose_rate 	15
combos = np.array(
	np.meshgrid(
		[-1, 1, 2, 3, -4, 4, 5, 6, -7, 7, 8, 9, -10, 10, 11, 12, -13, 13, 14, 15],
)).T.reshape(-1, 1)
print('1 параметр', len(combos)) 

combos = np.array(
	np.meshgrid(
		[-1, 1, 2, 3, -4, 4, 5, 6, -7, 7, 8, 9, -10, 10, 11, 12, -13, 13, 14, 15],
		[-1, 1, 2, 3, -4, 4, 5, 6, -7, 7, 8, 9, -10, 10, 11, 12, -13, 13, 14, 15],
)).T.reshape(-1, 2)
print('2 параметра', len(combos)) 

result = np.empty((0, 3))
combos = np.array(
	np.meshgrid(
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
)).T.reshape(-1, 3)
print('3 параметра', len(combos)) 
result = np.concatenate((result, combos))
print(len(result), result)
combos = np.array(
	np.meshgrid(
		[2, 5, 8, 11, 14], 
		[-3, -6, -9, -12, -15],
		[1, 4, 7, 10, 13, -1, -4, -7, -10, -13], 
)).T.reshape(-1, 3)
print('3 параметра, по 1 из draw|loose|win', len(combos)) 
result = np.concatenate((result, combos))
print(len(result), result)
result = np.unique(result, axis=0)
print(len(result), result)

1 параметр 20
2 параметра 400
3 параметра 8000
8000 [[ -1.  -1.  -1.]
 [ -1.   1.  -1.]
 [ -1.   2.  -1.]
 ...
 [-15.  13. -15.]
 [-15.  14. -15.]
 [-15. -15. -15.]]
3 параметра, по 1 из draw|loose|win 250
8250 [[ -1.  -1.  -1.]
 [ -1.   1.  -1.]
 [ -1.   2.  -1.]
 ...
 [ 14.  -9. -13.]
 [ 14. -12. -13.]
 [ 14. -15. -13.]]
8000 [[-15. -15. -15.]
 [-15. -15. -13.]
 [-15. -15. -12.]
 ...
 [ 14.  14.  11.]
 [ 14.  14.  13.]
 [ 14.  14.  14.]]


In [109]:
import numpy as np
import pandas as pd
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
from matplotlib.table import Table
from random import choice

In [158]:
data = {}
for name in ['no_njit', 'prange', 'range']:
	data[name] = {'iterations': [], 'times': [], 'avg': 0, 'weight_avg': 0}
	with open(f'{name}_log.log') as file:
		for line in file.readlines():
			iteration, minutes, seconds = [int(num) for num in line.split(',')]
			data[name]['iterations'].append(iteration)
			data[name]['times'].append(minutes*60 + seconds)
	count = len(data[name]['times'])
	data[name]['avg'] = sum(data[name]['times']) / count
	data[name]['weight_avg'] = sum([data[name]['times'][i] / data[name]['iterations'][i] for i in range(count)]) / count

print(
	'Имя		'
	'Среднее           ', 
	'Средневзвешенное  ', 
	'Среднее %         ', 
	'Средневзвешенное %',
	sep='\t'
)
for name in data:
	print(
		f'{name:10}',
		data[name]['avg'], 
		data[name]['weight_avg'], 
		
		data[name]['avg'] / sum([data[i]['avg'] for i in data]) * 100, 
		data[name]['weight_avg'] / sum([data[i]['weight_avg'] for i in data]) * 100, 
		sep=' \t'
	)
# for i in range(2, 1_320, 10):
# 	plt.figure()
# 	for name in ['no_njit', 'prange', 'range']:
# 		n = len([*filter(lambda x: x < i, data[name]['times'])]) + 1
# 		plt.plot(data[name]['iterations'][:n], data[name]['times'][:n], label=name)

# 	plt.legend(bbox_to_anchor=(1.05, .8), loc='center left')
# 	plt.ylabel('time (seconds)')
# 	plt.xlabel('iterations')
# 	plt.show()

Имя		Среднее           	Средневзвешенное  	Среднее %         	Средневзвешенное %
no_njit    	655.3921568627451 	0.25678765827149974 	40.323606024693 	92.85851868590889
prange     	483.4945054945055 	0.009858313183708618 	29.747444717662663 	3.5649235058371573
range      	486.4445554445554 	0.009890486271493273 	29.928949257644348 	3.5765578082539653
